In [1]:
import os
import numpy as np
import pandas as pd
from itertools import product
from tqdm import tqdm
from iteration_utilities import unique_everseen

from data_generator import DataGenerator
from components import Components

In [2]:
# generator combinations of different components
com = Components()
print(com.gym())

gyms_comb = list(product(*list(com.gym().values())))
keys = list(com.gym().keys())
gyms = []

for _ in tqdm(gyms_comb):
    gym = {}
    for j, __ in enumerate(_):
        gym[keys[j]] = __
        
    if gym['layers'] != len(gym['hidden_size_list']):
        continue
        
    # delete ResNet & FTT ReLU: activation layers
    if gym['network_architecture'] in ['ResNet', 'FTT']:
        if gym['act_fun'] != 'ReLU':
            continue

    # delete FTT: hidden_size_list, drop out
    if gym['network_architecture'] == 'FTT':
        gym['hidden_size_list'] = None
        gym['dropout'] = None
        
    gyms.append(gym)
    
print(len(gyms))

{'augmentation': [None], 'preprocess': ['minmax', 'normalize'], 'network_architecture': ['MLP', 'AE', 'ResNet', 'FTT'], 'layers': [1, 2, 3], 'hidden_size_list': [[20], [100, 20], [100, 50, 20]], 'act_fun': ['Tanh', 'ReLU', 'LeakyReLU'], 'dropout': [0.0, 0.1, 0.3], 'training_strategy': [None], 'loss_name': ['minus', 'inverse', 'hinge', 'deviation'], 'optimizer_name': ['SGD', 'Adam', 'RMSprop'], 'batch_resample': [True], 'epochs': [20, 50, 100], 'batch_size': [16, 64, 256], 'lr': [0.01, 0.001], 'weight_decay': [0.01, 0.0001]}


100%|██████████████████████████████████████████████████████████████████████| 279936/279936 [00:00<00:00, 332171.63it/s]

62208


In [3]:
# random selection
idx = np.random.choice(np.arange(len(gyms)), 100, replace=False)
gyms = [gyms[_] for _ in idx]
print(len(gyms))


gyms = list(unique_everseen(gyms))
print(len(gyms))

100
100


In [4]:
gyms[0]

{'augmentation': None,
 'preprocess': 'minmax',
 'network_architecture': 'ResNet',
 'layers': 2,
 'hidden_size_list': [100, 20],
 'act_fun': 'ReLU',
 'dropout': 0.1,
 'training_strategy': None,
 'loss_name': 'inverse',
 'optimizer_name': 'SGD',
 'batch_resample': True,
 'epochs': 50,
 'batch_size': 64,
 'lr': 0.01,
 'weight_decay': 0.01}

Experiments

In [5]:
dataset_list = [os.path.splitext(_)[0] for _ in os.listdir('datasets') if os.path.splitext(_)[1] == '.npz']

df_results_AUCROC = pd.DataFrame(data=None, index=[str(_) for _ in gyms], columns=dataset_list)
df_results_AUCPR = pd.DataFrame(data=None, index=[str(_) for _ in gyms], columns=dataset_list)

In [ ]:
for dataset in dataset_list:
    # generate data
    data_generator = DataGenerator(dataset=dataset)
    data = data_generator.generator(la=0.20)
    
    for gym in tqdm(gyms):
        com = Components(data=data,
                         augmentation=gym['augmentation'],
                         preprocess=gym['preprocess'],
                         network_architecture=gym['network_architecture'],
                         layers=gym['layers'],
                         hidden_size_list=gym['hidden_size_list'],
                         act_fun=gym['act_fun'],
                         dropout=gym['dropout'],
                         training_strategy=gym['training_strategy'],
                         loss_name=gym['loss_name'],
                         optimizer_name=gym['optimizer_name'],
                         batch_resample=gym['batch_resample'],
                         epochs=gym['epochs'],
                         batch_size=gym['batch_size'],
                         lr=gym['lr'],
                         weight_decay=gym['weight_decay'])

        try:
            com.f_train()
            metrics = com.f_predict_score()

            df_results_AUCROC.loc[str(gym), dataset] = metrics['aucroc']
            df_results_AUCPR.loc[str(gym), dataset] = metrics['aucpr']

            print(f'Dataset: {dataset}, Current combination: {gym}, training sucessfully.')
            df_results_AUCROC.to_csv('result_AUCROC.csv', index=True)
            df_results_AUCPR.to_csv('result_AUCPR.csv', index=True)

        except Exception as e:
            print(f'Dataset: {dataset}, Current combination: {gym}, training failure...Error: {e}')
            pass
            continue

subsampling for dataset 10_cover...


  1%|▊                                                                                 | 1/100 [00:28<47:09, 28.58s/it]

Dataset: 10_cover, Current combination: {'augmentation': None, 'preprocess': 'minmax', 'network_architecture': 'ResNet', 'layers': 2, 'hidden_size_list': [100, 20], 'act_fun': 'ReLU', 'dropout': 0.1, 'training_strategy': None, 'loss_name': 'inverse', 'optimizer_name': 'SGD', 'batch_resample': True, 'epochs': 50, 'batch_size': 64, 'lr': 0.01, 'weight_decay': 0.01}, training failure...Error: Input contains NaN, infinity or a value too large for dtype('float32').


  2%|█▋                                                                                | 2/100 [00:44<34:44, 21.27s/it]

Dataset: 10_cover, Current combination: {'augmentation': None, 'preprocess': 'minmax', 'network_architecture': 'MLP', 'layers': 2, 'hidden_size_list': [100, 20], 'act_fun': 'ReLU', 'dropout': 0.0, 'training_strategy': None, 'loss_name': 'hinge', 'optimizer_name': 'SGD', 'batch_resample': True, 'epochs': 50, 'batch_size': 64, 'lr': 0.001, 'weight_decay': 0.01}, training sucessfully.


...

Dataset: 10_cover, Current combination: {'augmentation': None, 'preprocess': 'normalize', 'network_architecture': 'MLP', 'layers': 2, 'hidden_size_list': [100, 20], 'act_fun': 'ReLU', 'dropout': 0.1, 'training_strategy': None, 'loss_name': 'hinge', 'optimizer_name': 'Adam', 'batch_resample': True, 'epochs': 20, 'batch_size': 16, 'lr': 0.01, 'weight_decay': 0.0001}, training sucessfully.
GPU is off


 34%|█████████████████████████▊                                                  | 34/100 [1:12:28<5:34:34, 304.16s/it]

Dataset: 10_cover, Current combination: {'augmentation': None, 'preprocess': 'normalize', 'network_architecture': 'FTT', 'layers': 2, 'hidden_size_list': None, 'act_fun': 'ReLU', 'dropout': None, 'training_strategy': None, 'loss_name': 'minus', 'optimizer_name': 'RMSprop', 'batch_resample': True, 'epochs': 50, 'batch_size': 64, 'lr': 0.01, 'weight_decay': 0.01}, training sucessfully.


 35%|██████████████████████████▌                                                 | 35/100 [1:12:36<3:53:24, 215.46s/it]

Dataset: 10_cover, Current combination: {'augmentation': None, 'preprocess': 'minmax', 'network_architecture': 'MLP', 'layers': 1, 'hidden_size_list': [20], 'act_fun': 'ReLU', 'dropout': 0.0, 'training_strategy': None, 'loss_name': 'minus', 'optimizer_name': 'RMSprop', 'batch_resample': True, 'epochs': 50, 'batch_size': 256, 'lr': 0.001, 'weight_decay': 0.0001}, training sucessfully.
GPU is off
